# Cours "Géomatique" - Géocodage
### Louis Maritaud
### louis.maritaud@unilim.fr

## Objectifs pédagogiques
- Comprendre le géocodage et le géocodage inverse
- Utiliser geopy pour transformer des adresses en coordonnées
- Gérer les erreurs et les limites des services de géocodage
- Intégrer géocodage, GeoPandas et Folium dans un projet complet
- Créer une application cartographique interactive

# Résumé de ce qu'on a vu

## GeoPandas et les GeoDataFrame
GeoPandas permet de manipuler des données géographiques. Un **GeoDataFrame** contient une colonne `geometry` avec les informations spatiales.

**Syntaxe de base :**
```python
import geopandas as gpd

# Lire un fichier géographique
gdf = gpd.read_file("fichier.geojson")

# Reprojeter en WGS84
gdf = gdf.to_crs(epsg=4326)
```

## Folium pour les cartes interactives
Folium permet de créer des cartes web interactives :

```python
import folium

# Créer une carte
carte = folium.Map(location=[45.85, 1.25], zoom_start=13)

# Ajouter un marqueur
folium.Marker(
    location=[45.85, 1.25],
    popup='Mon lieu',
    icon=folium.Icon(color='red')
).add_to(carte)

# Afficher
carte
```

## Opérations spatiales essentielles
```python
# Buffer (zone tampon)
gdf['geometry'] = gdf.buffer(500)  # 500 mètres

# Intersection spatiale
points_dans_zone = gpd.sjoin(points_gdf, zone_gdf, predicate='within')
```

# Le géocodage, c'est quoi ?

Le **géocodage** est le processus qui consiste à transformer une **adresse textuelle** en **coordonnées géographiques** (latitude, longitude).

**Exemple :**
- Entrée : `"33 rue François Mitterrand, 87000 Limoges"`
- Sortie : `latitude = 45.8336, longitude = 1.2611`

Le **géocodage inverse** fait l'opération contraire : il transforme des coordonnées en adresse.

**Exemple :**
- Entrée : `latitude = 45.8336, longitude = 1.2611`
- Sortie : `"33 rue François Mitterrand, 87000 Limoges"`

## Pourquoi c'est utile ?

Le géocodage est indispensable quand vos données contiennent des adresses mais pas de coordonnées :
- Liste de clients à cartographier
- Adresses de services publics (écoles, mairies, bibliothèques)
- Lieux d'événements à visualiser
- Points de collecte ou de livraison

**Sans géocodage**, placer ces points sur une carte interactive est extrêmement long !

## Les services de géocodage

Il existe plusieurs services de géocodage :

1. **Nominatim (OpenStreetMap)** - Recommandé pour débuter
   - Gratuit et open source
   - Bonne qualité pour la France
   - Limite : 1 requête par seconde

2. **Google Geocoding API**
   - Très précis
   - Payant après 40 000 requêtes/mois
   - Nécessite une clé API

3. **API Adresse (data.gouv.fr)**
   - Spécifique à la France
   - Gratuit et illimité
   - Très bon pour les adresses françaises

Dans ce cours, nous utiliserons **Nominatim** car il est gratuit, simple d'utilisation et ne nécessite pas de clé API.

### Installation

```python
# À exécuter une seule fois dans votre terminal
pip install geopy
```

# Géocodage simple avec geopy

## Premier géocodage

Commençons par géocoder une seule adresse pour comprendre le principe.

In [15]:
from geopy.geocoders import Nominatim
import time

# Créer un géocodeur
# Le user_agent est obligatoire, c'est comme une signature
geolocator = Nominatim(user_agent="cours_geomatique")

# Géocoder une adresse
adresse = "33 rue François Mitterrand, 87000 Limoges"
location = geolocator.geocode(adresse)

# Afficher le résultat
print(f"Adresse trouvée : {location.address}")
print(f"Latitude : {location.latitude}")
print(f"Longitude : {location.longitude}")

Adresse trouvée : Université de Limoges, 33, Rue François Mitterrand, Le Château, Clos Moreau, Limoges, Haute-Vienne, Nouvelle-Aquitaine, France métropolitaine, 87000, France
Latitude : 45.8260427
Longitude : 1.2585756


**Ce qui se passe ici :**
1. On crée un objet `Nominatim` avec un `user_agent` (obligatoire, c'est notre identifiant)
2. On utilise la méthode `.geocode()` avec notre adresse
3. On récupère un objet `location` qui contient les coordonnées

**Astuce :** L'adresse retournée peut être légèrement différente de celle entrée. Nominatim "nettoie" et standardise l'adresse.

## Géocodage inverse

Maintenant, faisons l'inverse : trouvons l'adresse à partir de coordonnées.

In [16]:
# Coordonnées de la gare de Limoges
latitude = 45.8336
longitude = 1.2611

# Géocodage inverse
location = geolocator.reverse(f"{latitude}, {longitude}")

print(f"Adresse trouvée : {location.address}")
print(f"Code postal : {location.raw.get('address', {}).get('postcode', 'Non trouvé')}")

Adresse trouvée : Renault Minute, 23, Avenue Garibaldi, Le Château, Le Grand-Treuil, Limoges, Haute-Vienne, Nouvelle-Aquitaine, France métropolitaine, 87085, France
Code postal : 87085


**Note importante :** Le format pour `.reverse()` est une chaîne `"latitude, longitude"` (pas de liste Python !).

L'objet `location.raw` contient beaucoup d'informations détaillées (rue, ville, code postal, etc.).

## 1.3 Géocoder plusieurs adresses

Dans la pratique, on a souvent plusieurs adresses à géocoder. **Attention à la limite de 1 requête par seconde !**

In [28]:
# Liste d'adresses de bibliothèques à Limoges
adresses = [
    "2 Place Aimé Césaire, Limoges",
    "2 Rue du Général de Gaulle, Isle",
    "2 Rue du Docteur Raymond Marcland, Limoges",
    "123 avenue Albert Thomas, Limoges",
    "39C Rue Camille Guérin, Limoges"
]


# Stocker les résultats
resultats = []

for adresse in adresses:
    try:
        location = geolocator.geocode(adresse)
        
        if location:
            resultats.append({
                'adresse_originale': adresse,
                'adresse_trouvee': location.address,
                'latitude': location.latitude,
                'longitude': location.longitude
            })
            print(f"Géocodé : {adresse[:50]}...")
        else:
            print(f"Non trouvé : {adresse}")
    
    except Exception as e:
        print(f"Erreur : {adresse} - {e}")
    
    # IMPORTANT : respecter la limite de 1 requête/seconde
    time.sleep(1)

print(f"\n{len(resultats)} adresses géocodées sur {len(adresses)}")

Géocodé : 2 Place Aimé Césaire, Limoges...
Géocodé : 2 Rue du Général de Gaulle, Isle...
Géocodé : 2 Rue du Docteur Raymond Marcland, Limoges...
Géocodé : 123 avenue Albert Thomas, Limoges...
Géocodé : 39C Rue Camille Guérin, Limoges...

5 adresses géocodées sur 5


**Points clés :**
1. On utilise un **try/except** pour gérer les erreurs (service indisponible, timeout, etc.)
2. On teste si `location` existe (certaines adresses peuvent ne pas être trouvées)
3. On respecte la limite avec `time.sleep(1)` - **très important !**

**Si vous ne respectez pas cette limite**, Nominatim vous bloquera temporairement.

## 1.4 Transformer en GeoDataFrame

Maintenant qu'on a nos coordonnées, créons un GeoDataFrame pour pouvoir faire des analyses spatiales.

In [29]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Créer un DataFrame pandas classique
df = pd.DataFrame(resultats)

# Créer une colonne geometry avec des objets Point
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]

# Convertir en GeoDataFrame
gdf_bibliotheques = gpd.GeoDataFrame(
    df, 
    geometry=geometry,
    crs='EPSG:4326'  # WGS84, le système de Nominatim
)

# Afficher
print(gdf_bibliotheques[['adresse_originale', 'latitude', 'longitude']])

                            adresse_originale   latitude  longitude
0               2 Place Aimé Césaire, Limoges  45.826133   1.259288
1            2 Rue du Général de Gaulle, Isle  45.805389   1.226094
2  2 Rue du Docteur Raymond Marcland, Limoges  45.813378   1.234940
3           123 avenue Albert Thomas, Limoges  45.837936   1.238617
4             39C Rue Camille Guérin, Limoges  45.819775   1.230249


**Astuce :** Remarquez qu'on crée les Points avec `(longitude, latitude)` et non `(latitude, longitude)` ! C'est l'ordre standard en géomatique (X, Y).

## 1.5 Visualiser sur une carte Folium

In [30]:
import folium

# Créer une carte centrée sur Limoges
carte = folium.Map(
    location=[45.85, 1.25],
    zoom_start=13,
    tiles='OpenStreetMap'
)

# Ajouter chaque bibliothèque comme marqueur
for idx, row in gdf_bibliotheques.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"<b>{row['adresse_originale']}</b><br>{row['adresse_trouvee']}",
        tooltip=row['adresse_originale'],
        icon=folium.Icon(color='blue', icon='book', prefix='fa')
    ).add_to(carte)

carte

# Partie 2 : Projet intégrateur - Accessibilité des services publics

Maintenant qu'on maîtrise le géocodage, créons un projet complet qui mobilise tout ce qu'on a appris :
- Géocodage avec geopy
- Analyse spatiale avec GeoPandas (buffers, intersections)
- Visualisation interactive avec Folium

**Objectif du projet :** Analyser l'accessibilité des bibliothèques à Limoges en identifiant les zones à moins de 500m d'une bibliothèque.

## Étape 1 : Préparer les données

Créons une liste plus complète de bibliothèques et géocodons-les.

In [31]:
# Liste étendue de bibliothèques et médiathèques à Limoges
bibliotheques = [
    {"nom": "BFM Limoges", "adresse": "2 Place Aimé Césaire, Limoges"},
    {"nom": "Médiathèque Municipale", "adresse": "2 Rue du Général de Gaulle, Isle"},
    {"nom": "Bibliothèque Universitaire de Santé", "adresse": "2 Rue du Docteur Raymond Marcland, Limoges"},
    {"nom": "Bibliothèque des Sciences et techniques de l'Université de Limoges", "adresse": "123 avenue Albert Thomas, Limoges"},
    {"nom": "BU de Lettres et Sciences Humaines", "adresse": "39C Rue Camille Guérin, Limoges"}
]

# Géocoder chaque bibliothèque
bibliotheques_geocodees = []

for bib in bibliotheques:
    try:
        location = geolocator.geocode(bib['adresse'])
        
        if location:
            bibliotheques_geocodees.append({
                'nom': bib['nom'],
                'adresse': bib['adresse'],
                'latitude': location.latitude,
                'longitude': location.longitude
            })
            print(f"{bib['nom']}")
        else:
            print(f"{bib['nom']} - adresse non trouvée")
    
    except Exception as e:
        print(f"{bib['nom']} - erreur : {e}")
    
    time.sleep(1)

print(f"\n{len(bibliotheques_geocodees)} bibliothèques géocodées")

BFM Limoges
Médiathèque Municipale
Bibliothèque Universitaire de Santé
Bibliothèque des Sciences et techniques de l'Université de Limoges
BU de Lettres et Sciences Humaines

5 bibliothèques géocodées


## Étape 2 : Créer le GeoDataFrame et les zones de couverture

On va créer des zones tampons de 500m autour de chaque bibliothèque pour représenter leur zone d'accessibilité.

In [32]:
# Créer le GeoDataFrame
df_bib = pd.DataFrame(bibliotheques_geocodees)
geometry = [Point(xy) for xy in zip(df_bib['longitude'], df_bib['latitude'])]
gdf_bib = gpd.GeoDataFrame(df_bib, geometry=geometry, crs='EPSG:4326')

# IMPORTANT : Reprojeter en Lambert 93 pour faire des calculs en mètres
gdf_bib_lambert = gdf_bib.to_crs(epsg=2154)

# Créer des buffers de 500m (en Lambert 93)
gdf_bib_lambert['zone_500m'] = gdf_bib_lambert.geometry.buffer(500)

# Reprojeter en WGS84 pour Folium
gdf_zones = gdf_bib_lambert.copy()
gdf_zones = gdf_zones.set_geometry('zone_500m')
gdf_zones = gdf_zones.to_crs(epsg=4326)

print(f"Zones de couverture créées pour {len(gdf_zones)} bibliothèques")

Zones de couverture créées pour 5 bibliothèques


**Pourquoi ces projections ?**
1. On part en **EPSG:4326** (WGS84) car c'est le système de Nominatim
2. On reprojete en **EPSG:2154** (Lambert 93) pour calculer les buffers en **mètres**
3. On reprojete en **EPSG:4326** pour Folium

**Astuce :** Un buffer de 500 en EPSG:4326 ne ferait pas 500 mètres, mais 500 degrés !

## Créer la carte interactive finale

Créons une carte avec :
- Les bibliothèques (marqueurs)
- Les zones de couverture de 500m (polygones)
- Des popups informatifs

In [33]:
# Créer la carte
carte_finale = folium.Map(
    location=[45.85, 1.25],
    zoom_start=13,
    tiles='OpenStreetMap'
)

# Ajouter les zones de couverture (polygones)
for idx, row in gdf_zones.iterrows():
    folium.GeoJson(
        row['zone_500m'],
        style_function=lambda x: {
            'fillColor': 'lightblue',
            'color': 'blue',
            'weight': 2,
            'fillOpacity': 0.3
        },
        tooltip=f"Zone de 500m autour de {row['nom']}"
    ).add_to(carte_finale)

# Ajouter les marqueurs des bibliothèques
for idx, row in gdf_bib.iterrows():
    popup_html = f"""
    <div style="width: 200px;">
        <h4>{row['nom']}</h4>
        <p><b>Adresse :</b> {row['adresse']}</p>
        <p><b>Coordonnées :</b><br>
        Lat: {row['latitude']:.4f}<br>
        Lon: {row['longitude']:.4f}</p>
    </div>
    """
    
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_html, max_width=250),
        tooltip=row['nom'],
        icon=folium.Icon(color='red', icon='book', prefix='fa')
    ).add_to(carte_finale)

# Ajouter un titre
titre_html = '''
             <div style="position: fixed; 
             top: 10px; left: 50px; width: 400px; height: 90px; 
             background-color: white; border:2px solid grey; z-index:9999; 
             font-size:14px; padding: 10px">
             <h4>Accessibilité des bibliothèques à Limoges</h4>
             <p>Les zones bleues représentent un rayon de 500m autour de chaque bibliothèque</p>
             </div>
             '''
carte_finale.get_root().html.add_child(folium.Element(titre_html))

carte_finale

## Étape 4 : Analyse avancée - Calculer la surface couverte

Calculons quelle surface totale est accessible à moins de 500m d'une bibliothèque.

In [34]:
# Fusionner toutes les zones de couverture
# (en Lambert 93 pour avoir une surface en m²)
from shapely.ops import unary_union

# Union de toutes les zones
zone_totale = unary_union(gdf_bib_lambert['zone_500m'])

# Calculer la surface en km²
surface_km2 = zone_totale.area / 1_000_000

print(f"Surface totale couverte : {surface_km2:.2f} km²")
print(f"Nombre de bibliothèques : {len(gdf_bib)}")
print(f"Surface moyenne par bibliothèque : {surface_km2/len(gdf_bib):.2f} km²")

Surface totale couverte : 3.84 km²
Nombre de bibliothèques : 5
Surface moyenne par bibliothèque : 0.77 km²


## Étape 5 : Sauvegarder la carte

Exportons notre carte pour la partager ou l'intégrer dans un site web.

In [35]:
# Sauvegarder en HTML
carte_finale.save('accessibilite_bibliotheques_limoges.html')
print("Carte sauvegardée : accessibilite_bibliotheques_limoges.html")

Carte sauvegardée : accessibilite_bibliotheques_limoges.html


# Pour aller plus loin

## Idées d'extensions pour votre projet

1. **Ajouter d'autres services publics**
   - Écoles primaires
   - Mairies annexes
   - Centres de santé
   - Parcs publics

2. **Analyser les superpositions**
   - Quelles zones sont couvertes par plusieurs services ?
   - Y a-t-il des "déserts" de services publics ?

3. **Intégrer des données démographiques**
   - Combien d'habitants vivent à moins de 500m d'une bibliothèque ?
   - Utiliser les données INSEE des IRIS

4. **Améliorer la visualisation**
   - Ajouter des clusters de marqueurs
   - Créer des cartes choroplèthes
   - Utiliser différentes couleurs par type de service

5. **Optimiser les distances**
   - Calculer les distances réelles de marche (avec OSRM ou GraphHopper)
   - Tenir compte du réseau routier

## Exemple d'extension : Ajouter des écoles

In [36]:
# Liste d'écoles primaires à Limoges (exemple)
ecoles = [
    {"nom": "École élémentaire Jules Ferry", "adresse": "Rue Jules Ferry, Limoges"},
    {"nom": "École Montmailler", "adresse": "Rue Montmailler, Limoges"},
    {"nom": "École primaire Beaupeyrat", "adresse": "Avenue Beaupeyrat, Limoges"}
]

# Géocoder les écoles
ecoles_geocodees = []
for ecole in ecoles:
    try:
        location = geolocator.geocode(ecole['adresse'])
        if location:
            ecoles_geocodees.append({
                'nom': ecole['nom'],
                'adresse': ecole['adresse'],
                'latitude': location.latitude,
                'longitude': location.longitude
            })
            print(f"{ecole['nom']}")
    except Exception as e:
        print(f"{ecole['nom']} - erreur : {e}")
    time.sleep(1)

# Créer le GeoDataFrame des écoles
if ecoles_geocodees:
    df_ecoles = pd.DataFrame(ecoles_geocodees)
    geometry_ecoles = [Point(xy) for xy in zip(df_ecoles['longitude'], df_ecoles['latitude'])]
    gdf_ecoles = gpd.GeoDataFrame(df_ecoles, geometry=geometry_ecoles, crs='EPSG:4326')
    
    # Ajouter à la carte
    for idx, row in gdf_ecoles.iterrows():
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=row['nom'],
            tooltip=row['nom'],
            icon=folium.Icon(color='green', icon='graduation-cap', prefix='fa')
        ).add_to(carte_finale)
    
    print(f"\n{len(gdf_ecoles)} écoles ajoutées à la carte")

carte_finale

École élémentaire Jules Ferry
École Montmailler
École primaire Beaupeyrat

3 écoles ajoutées à la carte


# Récapitulatif des commandes

## Géocodage avec geopy

```python
from geopy.geocoders import Nominatim
import time

# Créer un géocodeur
geolocator = Nominatim(user_agent="mon_application")

# Géocoder une adresse
location = geolocator.geocode("adresse")
latitude = location.latitude
longitude = location.longitude

# Géocodage inverse
location = geolocator.reverse("45.85, 1.25")
adresse = location.address

# IMPORTANT : Toujours respecter la limite de 1 requête/seconde
time.sleep(1)
```

## Créer un GeoDataFrame depuis des coordonnées

```python
from shapely.geometry import Point
import geopandas as gpd
import pandas as pd

# Créer des Points
geometry = [Point(lon, lat) for lon, lat in zip(df['longitude'], df['latitude'])]

# Créer le GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')
```

## Buffers et projections

```python
# Reprojeter en Lambert 93 pour des calculs en mètres
gdf_lambert = gdf.to_crs(epsg=2154)

# Créer un buffer de 500m
gdf_lambert['buffer_500m'] = gdf_lambert.geometry.buffer(500)

# Reprojeter en WGS84 pour Folium
gdf_wgs = gdf_lambert.to_crs(epsg=4326)
```

## Gestion des erreurs

```python
try:
    location = geolocator.geocode(adresse)
    if location:
        # Traiter le résultat
        pass
    else:
        print("Adresse non trouvée")
except Exception as e:
    print(f"Erreur : {e}")
```

# Exercices pratiques

## Exercice 1 : Géocoder vos lieux préférés
Créez une liste de 5 lieux que vous aimez à Limoges (restaurants, parcs, musées...), géocodez-les et créez une carte interactive personnalisée.

## Exercice 2 : Analyse de zones de livraison
Imaginez que vous gérez une entreprise de livraison. Vous avez 3 entrepôts à Limoges. Créez une carte montrant :
- Les emplacements des entrepôts
- Les zones de livraison de 1km autour de chaque entrepôt
- Identifiez les zones couvertes par plusieurs entrepôts

## Exercice 3 : Trouver le centre
À partir d'une liste d'adresses de vos amis, trouvez le point central optimal pour organiser une rencontre (centroïde des positions géocodées).

## Exercice 4 : API BAN (data.gouv.fr)
Recherchez comment utiliser l'API BAN française pour géocoder. Comparez les résultats avec Nominatim pour quelques adresses françaises.

## Projet final suggéré
Créez une application d'analyse d'accessibilité pour un type de service public de votre choix (pharmacies, arrêts de bus, parcs...) :
1. Récupérez les adresses (recherche web ou données ouvertes)
2. Géocodez-les
3. Créez des zones de couverture
4. Analysez les zones mal desservies
5. Proposez des emplacements optimaux pour de nouveaux services

# Ressources utiles

## Documentation
- **geopy :** https://geopy.readthedocs.io/
- **Nominatim :** https://nominatim.org/
- **API Adresse BAN :** https://adresse.data.gouv.fr/api-doc/adresse
- **GeoPandas :** https://geopandas.org/
- **Folium :** https://python-visualization.github.io/folium/

## Données ouvertes
- **data.gouv.fr :** Données publiques françaises
- **OpenStreetMap :** https://www.openstreetmap.org/
- **INSEE :** Données démographiques et géographiques

## Conseils importants

1. **Rate limiting** : Respectez toujours les limites des services (1 req/sec pour Nominatim)
2. **Cache** : Sauvegardez vos résultats de géocodage pour ne pas refaire les mêmes requêtes
3. **Qualité des adresses** : Plus l'adresse est précise, meilleur sera le géocodage
4. **Projections** : N'oubliez jamais de vérifier et convertir le CRS selon vos besoins
5. **Gestion d'erreurs** : Toujours utiliser try/except et vérifier si le résultat existe
6. **Performance** : Pour de gros volumes, utilisez des APIs professionnelles ou des solutions locales (Photon, Pelias)

## Alternative : Géocodage par lot

Pour géocoder beaucoup d'adresses à la fois, l'API BAN permet des requêtes en lot :

```python
import requests

url = "https://api-adresse.data.gouv.fr/search/csv/"
files = {'data': open('adresses.csv', 'rb')}
response = requests.post(url, files=files)
```

Cela permet de géocoder des milliers d'adresses très rapidement !